In [11]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import lower, col, explode, split, regexp_replace

In [2]:
spark_session = SparkSession.builder\
    .master("spark://192.168.2.70:7077") \
    .appName("Sepehr_3A")\
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
    .config("spark.shuffle.service.enabled", True)\
    .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
    .config("spark.cores.max", 4)\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/15 16:53:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# create spark contex
sc = spark_session.sparkContext

# create RDD from the textfile transcript in English
rdd_en = sc.textFile("hdfs://192.168.2.70:9000/europarl/europarl-v7.sv-en.en")

In [6]:
def prp(rdd):
    rdd = rdd.map(lambda a: a.lower().split())
    return rdd

In [7]:
rdd_en_prp = prp(rdd_en)

In [8]:
print(f"10 Entries of the English Version:\n {rdd_en_prp.take(10)}")

10 Entries of the English Version:
 [['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', '

In [9]:
def n_freq(language, rdd, n):
    wordcount = rdd.flatMap(lambda line: line).map(lambda word: (word,1)).reduceByKey(lambda a,b: a+b)
    freq = wordcount.sortBy(lambda x: x[1], False).take(n)
    print(f'{n} most Frequent Words in {language} version:')
    for word, count in freq:
          print(f"{word}: {count}")

In [10]:
n_freq("English", rdd_en_prp,10)

10 most Frequent Words in English version:
the: 3498574
of: 1659884
to: 1539823
and: 1288620
in: 1086089
that: 797576
a: 773812
is: 758087
for: 534270
we: 522879
